In [2]:
import logging
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List

from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score, recall_score, roc_auc_score
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [3]:
DATA_DIR = Path("datasets")
MODULE = "BBB"
PRESENTATIONS = ["2013B", "2013J"]
CUTOFFS = [14, 21, 30, 45, 60, 90, 120, 150]
LOOK_AHEAD = 14
MODEL_PATH = "short_term_warning_model.pkl"

logging.basicConfig(level=logging.INFO, format="[%(asctime)s] %(levelname)s - %(message)s")


In [4]:
def compute_inactivity_streak(days_list: List[int], start_day: int, end_day: int) -> int:
    if not days_list:
        return end_day - start_day + 1
    active = set(days_list)
    streak, d = 0, end_day
    while d >= start_day and d not in active:
        streak += 1
        d -= 1
    return streak

def load_raw(data_dir: Path):
    return {
        "info": pd.read_csv(data_dir / "studentInfo.csv"),
        "reg": pd.read_csv(data_dir / "studentRegistration.csv"),
        "vle": pd.read_csv(data_dir / "studentVle.csv"),
    }

def build_short_term_dataset(raw: Dict[str, pd.DataFrame], cutoffs: List[int]):
    reg = raw["reg"][
        (raw["reg"]["code_module"] == MODULE) &
        (raw["reg"]["code_presentation"].isin(PRESENTATIONS))
    ].copy()

    reg["date_unregistration"] = pd.to_numeric(reg["date_unregistration"], errors="coerce").fillna(999)
    reg["date_registration"] = pd.to_numeric(reg["date_registration"], errors="coerce").fillna(0)

    reg_lookup = reg[["id_student", "date_registration", "date_unregistration"]].drop_duplicates()

    vle = raw["vle"].merge(reg_lookup[["id_student", "date_registration"]], on="id_student", how="inner")
    vle["date"] = pd.to_numeric(vle["date"], errors="coerce")
    vle["sum_click"] = pd.to_numeric(vle["sum_click"], errors="coerce").fillna(0)
    vle = vle.dropna(subset=["date"])
    vle["days_since_reg"] = vle["date"] - vle["date_registration"]

    all_samples = []

    for cutoff in cutoffs:
        current_batch = reg_lookup[reg_lookup["date_unregistration"] > cutoff].copy()

        # target: sẽ unreg trong (cutoff, cutoff+LOOK_AHEAD]
        current_batch["target"] = (
            (current_batch["date_unregistration"] > cutoff) &
            (current_batch["date_unregistration"] <= cutoff + LOOK_AHEAD)
        ).astype(int)

        vle_cum = vle[
            (vle["id_student"].isin(current_batch["id_student"])) &
            (vle["days_since_reg"] <= cutoff) &
            (vle["days_since_reg"] >= 0)
        ].copy()

        vle_win = vle_cum[vle_cum["days_since_reg"] >= (cutoff - 13)].copy()

        # Total features
        agg_total = vle_cum.groupby("id_student").agg(
            total_clicks=("sum_click", "sum"),
            active_days_total=("days_since_reg", "nunique"),
            last_active=("days_since_reg", "max"),
        ).reset_index()

        denom_total = max(cutoff, 1)
        agg_total["clicks_per_day_total"] = agg_total["total_clicks"] / denom_total
        agg_total["active_ratio_total"] = agg_total["active_days_total"] / denom_total
        agg_total["days_since_last_active"] = cutoff - agg_total["last_active"]
        agg_total["avg_clicks_per_active_day_total"] = (
            agg_total["total_clicks"] / agg_total["active_days_total"]
        ).replace([np.inf, -np.inf], np.nan).fillna(0)

        # 14-day window features
        agg_14 = vle_win.groupby("id_student").agg(
            clicks_last_14_days=("sum_click", "sum"),
            active_days_14=("days_since_reg", "nunique"),
        ).reset_index()

        agg_14["clicks_per_day_14"] = agg_14["clicks_last_14_days"] / 14
        agg_14["active_ratio_14"] = agg_14["active_days_14"] / 14

        # Trend/streak in window
        c_0_7 = vle_win[vle_win["days_since_reg"] <= (cutoff - 7)].groupby("id_student")["sum_click"].sum().reset_index(name="clicks_0_7")
        c_8_14 = vle_win[vle_win["days_since_reg"] > (cutoff - 7)].groupby("id_student")["sum_click"].sum().reset_index(name="clicks_8_14")
        c_last_7 = vle_cum[vle_cum["days_since_reg"] > (cutoff - 7)].groupby("id_student")["sum_click"].sum().reset_index(name="clicks_last_7_days")

        streak_df = vle_win.groupby("id_student")["days_since_reg"].apply(lambda x: list(set(x))).reset_index()
        streak_df["inactivity_streak_14"] = streak_df["days_since_reg"].apply(lambda x: compute_inactivity_streak(x, cutoff - 13, cutoff))

        merged = current_batch[["id_student", "target"]].merge(agg_total, on="id_student", how="left")
        merged = merged.merge(agg_14, on="id_student", how="left")
        merged = merged.merge(c_0_7, on="id_student", how="left")
        merged = merged.merge(c_8_14, on="id_student", how="left")
        merged = merged.merge(c_last_7, on="id_student", how="left")
        merged = merged.merge(streak_df[["id_student", "inactivity_streak_14"]], on="id_student", how="left")

        merged.iloc[:, 2:] = merged.iloc[:, 2:].fillna(0)

        merged["trend_click_14"] = merged["clicks_8_14"] - merged["clicks_0_7"]
        merged["ratio_click_14"] = (merged["clicks_8_14"] + 1) / (merged["clicks_0_7"] + 1)
        merged["days_elapsed_since_reg"] = cutoff

        # CLIP ratios to valid ranges (tránh bug kiểu Nest > 1)
        merged["active_ratio_total"] = merged["active_ratio_total"].clip(0, 1)
        merged["active_ratio_14"] = merged["active_ratio_14"].clip(0, 1)

        all_samples.append(merged)

    final_df = pd.concat(all_samples, ignore_index=True)

    feature_cols = [
        "days_elapsed_since_reg",
        "clicks_per_day_total", "active_ratio_total",
        "avg_clicks_per_active_day_total", "days_since_last_active",
        "clicks_last_14_days", "active_days_14",
        "clicks_per_day_14", "active_ratio_14",
        "clicks_last_7_days", "clicks_0_7", "clicks_8_14",
        "trend_click_14", "ratio_click_14",
        "inactivity_streak_14",
    ]
    return final_df, feature_cols

def clean_input_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # ép numeric + fill
    for c in out.columns:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.fillna(0)

    # clip ratios
    if "active_ratio_total" in out: out["active_ratio_total"] = out["active_ratio_total"].clip(0, 1)
    if "active_ratio_14" in out: out["active_ratio_14"] = out["active_ratio_14"].clip(0, 1)

    # clip streak [0,14]
    if "inactivity_streak_14" in out: out["inactivity_streak_14"] = out["inactivity_streak_14"].clip(0, 14)

    # không cho giá trị âm ở clicks
    for c in out.columns:
        if "click" in c:
            out[c] = out[c].clip(lower=0)

    # days_since_last_active >=0
    if "days_since_last_active" in out: out["days_since_last_active"] = out["days_since_last_active"].clip(lower=0)

    return out

def heuristic_risk(row: pd.Series) -> float:
    """
    Heuristic nhẹ để đảm bảo: ít hoạt động + streak cao -> risk cao.
    Trả về [0,1]
    """
    streak = row["inactivity_streak_14"] / 14.0                 # 0..1
    low_click14 = 1.0 - np.tanh(row["clicks_last_14_days"] / 10) # càng ít click -> càng cao
    last7_zero = 1.0 if row["clicks_last_7_days"] <= 1 else 0.0
    days_last = np.tanh(row["days_since_last_active"] / 5)       # 0..~1

    score = 0.45*streak + 0.30*low_click14 + 0.15*last7_zero + 0.10*days_last
    return float(np.clip(score, 0, 1))

In [5]:
raw = load_raw(DATA_DIR)
final_df, feature_cols = build_short_term_dataset(raw, CUTOFFS)

X = final_df[feature_cols]
y = final_df["target"].astype(int)
groups = final_df["id_student"]

X = clean_input_features(X)


In [6]:
from sklearn.metrics import precision_recall_curve

CANDIDATES = {
    "LogReg": Pipeline([
        ("pt", PowerTransformer()),
        ("clf", LogisticRegression(
            max_iter=3000,
            class_weight="balanced",
            C=2.0,
            solver="lbfgs"
        ))
    ]),
    "RF": ImbPipeline([
        ("smote", SMOTE(sampling_strategy=0.6, random_state=42)),
        ("clf", RandomForestClassifier(
            n_estimators=500,
            max_depth=10,
            min_samples_leaf=10,
            class_weight="balanced",
            random_state=42,
            n_jobs=-1
        ))
    ]),
}

def best_threshold_by_f1(y_true, proba):
    p, r, th = precision_recall_curve(y_true, proba)
    f1 = 2*p*r/(p+r+1e-12)
    # precision_recall_curve trả th có length = len(p)-1
    best_idx = np.nanargmax(f1[:-1])
    return float(th[best_idx]), float(f1[best_idx])

gkf = GroupKFold(n_splits=5)

best_name, best_f1 = None, -1
best_model, best_thr = None, 0.5

for name, model in CANDIDATES.items():
    fold_f1s = []
    fold_thrs = []

    for tr_idx, te_idx in gkf.split(X, y, groups):
        Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
        ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]

        model.fit(Xtr, ytr)

        # chọn threshold tối ưu trên train-fold
        proba_tr = model.predict_proba(Xtr)[:, 1]
        thr, _ = best_threshold_by_f1(ytr, proba_tr)

        # chấm trên test-fold với threshold đó
        proba_te = model.predict_proba(Xte)[:, 1]
        pred_te = (proba_te >= thr).astype(int)

        fold_f1s.append(f1_score(yte, pred_te))
        fold_thrs.append(thr)

    mean_f1 = float(np.mean(fold_f1s))
    mean_thr = float(np.median(fold_thrs))
    logging.info(f"{name}: MeanF1={mean_f1:.4f} | MedianThr={mean_thr:.4f}")

    if mean_f1 > best_f1:
        best_f1 = mean_f1
        best_name = name
        best_model = model
        best_thr = mean_thr

# fit full + save model + threshold
best_model.fit(X, y)
joblib.dump(
    {"model": best_model, "feature_cols": feature_cols, "threshold": best_thr},
    MODEL_PATH
)
logging.info(f"Saved: {best_name}, F1={best_f1:.4f}, threshold={best_thr:.4f}")


[2025-12-27 21:38:25,053] INFO - LogReg: MeanF1=0.0538 | MedianThr=0.5895
[2025-12-27 21:38:34,708] INFO - RF: MeanF1=0.0308 | MedianThr=0.5044
[2025-12-27 21:38:35,058] INFO - Saved: LogReg, F1=0.0538, threshold=0.5895


In [7]:
print("Positive rate:", y.mean())
print(y.value_counts())


Positive rate: 0.01702863732462041
target
0    25572
1      443
Name: count, dtype: int64


In [ ]:
def predict_risk(bundle, input_df: pd.DataFrame,
                 alpha_model: float = 0.55,
                 heur_boost: float = 1.15,
                 k: float = 8.0,      # độ dốc
                 c: float = 0.50):    # điểm giữa
    model = bundle["model"]
    cols = bundle["feature_cols"]

    df = input_df.copy()
    df = df.rename(columns={"student_id": "id_student"})
    df = df[cols]
    df = clean_input_features(df)

    proba = model.predict_proba(df)[:, 1]
    heur = df.apply(heuristic_risk, axis=1).values

    heur2 = np.clip(heur * heur_boost, 0, 1)
    raw = alpha_model * proba + (1 - alpha_model) * heur2

    # map raw -> display probability (0..1), giúp nhìn "đã"
    display = 1 / (1 + np.exp(-k * (raw - c)))

    return proba, heur, heur2, raw, display

# -------------------------
# QUICK TEST with your 3 samples
# -------------------------
bundle = joblib.load(MODEL_PATH)

samples = [
  {
    "student_id": "25",
    "days_elapsed_since_reg": 32,
    "clicks_per_day_total": 2.3125,
    "active_ratio_total": 0.71875,
    "avg_clicks_per_active_day_total": 3.217391304347826,
    "days_since_last_active": 0,
    "clicks_last_14_days": 53,
    "active_days_14": 14,
    "clicks_per_day_14": 3.7857142857142856,
    "active_ratio_14": 1.1428571428571428,  # will be clipped to 1
    "clicks_last_7_days": 19,
    "clicks_0_7": 10,
    "clicks_8_14": 43,
    "trend_click_14": 33,
    "ratio_click_14": 4,
    "inactivity_streak_14": 1
  },
  {
    "student_id": "59",
    "days_elapsed_since_reg": 26,
    "clicks_per_day_total": 1.6153846153846154,
    "active_ratio_total": 0.46153846153846156,
    "avg_clicks_per_active_day_total": 3.5,
    "days_since_last_active": 1,
    "clicks_last_14_days": 42,
    "active_days_14": 12,
    "clicks_per_day_14": 3,
    "active_ratio_14": 0.8571428571428571,
    "clicks_last_7_days": 4,
    "clicks_0_7": 8,
    "clicks_8_14": 34,
    "trend_click_14": 26,
    "ratio_click_14": 3.888888888888889,
    "inactivity_streak_14": 2
  },
  {
    "student_id": "60",
    "days_elapsed_since_reg": 31,
    "clicks_per_day_total": 0.16129032258064516,
    "active_ratio_total": 0.16129032258064516,
    "avg_clicks_per_active_day_total": 1,
    "days_since_last_active": 2,
    "clicks_last_14_days": 3,
    "active_days_14": 3,
    "clicks_per_day_14": 0.21428571428571427,
    "active_ratio_14": 0.21428571428571427,
    "clicks_last_7_days": 1,
    "clicks_0_7": 1,
    "clicks_8_14": 2,
    "trend_click_14": 1,
    "ratio_click_14": 1.5,
    "inactivity_streak_14": 6
  }
]
bundle = joblib.load(MODEL_PATH)
thr = bundle["threshold"]

inp = pd.DataFrame(samples)
proba, heur, heur2, raw, display = predict_risk(bundle, inp)

out = pd.DataFrame({
    "student_id": inp["student_id"].astype(str),
    "model_proba": proba,
    "heur_boosted": heur2,
    "raw_score": raw,
    "display_risk": display,
    "final_pred": (raw >= bundle["threshold"]).astype(int)  # dùng raw để quyết định
}).sort_values("display_risk", ascending=False)

print(out)



  student_id  model_proba  heur_boosted  raw_score  display_risk  final_pred
2         60     0.586206      0.551593   0.570631      0.637619           0
1         59     0.443094      0.095209   0.286545      0.153471           0
0         25     0.363012      0.035142   0.215471      0.093110           0


In [ ]:
bundle = joblib.load(MODEL_PATH)
thr = bundle["threshold"]

inp = pd.DataFrame(samples)
proba, heur, heur2, final = predict_risk(bundle, inp)

out = pd.DataFrame({
    "student_id": inp["student_id"].astype(str),
    "model_proba": proba,
    "heur": heur,
    "heur_boosted": heur2,
    "final_risk": final,
    "final_pred": (final >= thr).astype(int)
}).sort_values("final_risk", ascending=False)

print("Saved threshold:", thr)
print(out)


In [48]:
import logging
from datetime import datetime

logging.basicConfig(level=logging.INFO, format="[%(asctime)s] %(levelname)s - %(message)s")

# Fake email map (demo)
EMAIL_MAP = {
    25: "learner25@example.com",
    59: "learner59@example.com",
    60: "learner60@example.com",
}

def send_email_fake(to_email: str, subject: str, body: str):
    logging.info("")

def log_dropout_demo():
    # Fake prediction rates (you can change these numbers)
    fake_predictions = [
        {"student_id": 25, "name": "Người học A", "risk": 0.1215, "action": "SAFE"},
        {"student_id": 59, "name": "Người học B", "risk": 0.4234, "action": "REMIND"},
        {"student_id": 60, "name": "Người học C", "risk": 0.8121, "action": "ALERT"},
    ]

    logging.info("=== DROP-OUT EARLY WARNING (DEMO LOG) ===")
    logging.info(f"Run time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    for item in fake_predictions:
        sid = item["student_id"]
        name = item["name"]
        risk = item["risk"]
        action = item["action"]
        email = EMAIL_MAP.get(sid, f"learner{sid}@example.com")

        if action == "SAFE":
            logging.info(f"[{name} | id={sid}] Tỷ lệ dự đoán bỏ học: {risk:.2%} \n\t\t\t\t\t\t       -> ✅ SAN TOÀN (không gửi mail)\n")
        elif action == "REMIND":
            logging.info(f"[{name} | id={sid}] Tỷ lệ dự đoán bỏ học: {risk:.2%} \n\t\t\t\t\t\t       -> 🟡 CẦN NHẮC NHỞ (gửi mail nhắc học)\n")
           
        elif action == "ALERT":
            logging.info(f"[{name} | id={sid}] Tỷ lệ dự đoán bỏ học: {risk:.2%} \n\t\t\t\t\t\t       -> 🔴 NGUY CƠ CAO (gửi mail cảnh báo)\n")
           

    logging.info("=== END DEMO LOG ===")

# Run demo
log_dropout_demo()


[2025-12-27 22:58:46,793] INFO - === DROP-OUT EARLY WARNING (DEMO LOG) ===
[2025-12-27 22:58:46,794] INFO - Run time: 2025-12-27 22:58:46
[2025-12-27 22:58:46,795] INFO - [Người học A | id=25] Tỷ lệ dự đoán bỏ học: 12.15% 
						       -> ✅ SAN TOÀN (không gửi mail)

[2025-12-27 22:58:46,795] INFO - [Người học B | id=59] Tỷ lệ dự đoán bỏ học: 42.34% 
						       -> 🟡 CẦN NHẮC NHỞ (gửi mail nhắc học)

[2025-12-27 22:58:46,796] INFO - [Người học C | id=60] Tỷ lệ dự đoán bỏ học: 81.21% 
						       -> 🔴 NGUY CƠ CAO (gửi mail cảnh báo)

[2025-12-27 22:58:46,796] INFO - === END DEMO LOG ===
